In [3]:
import os
import urllib.request
import json
import pandas as pd
import time
import re
from konlpy.tag import Okt
import numpy as np
from sentence_transformers import SentenceTransformer, util

# 불러올 파일들의 공통 부분과 도시 목록 정의
cities = ['파리', '오사카', '방콕', '뉴욕']
categories = ['맛집', '관광', '핫플', '숙소', '쇼핑', '여행']

def getresult(client_id, client_secret, query, display=10, start=1, sort='sim'):
    encText = urllib.parse.quote(query)
    url = "https://openapi.naver.com/v1/search/blog?query=" + encText + \
          "&display=" + str(display) + "&start=" + str(start) + "&sort=" + sort

    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id", client_id)
    request.add_header("X-Naver-Client-Secret", client_secret)
    try:
        response = urllib.request.urlopen(request)
        rescode = response.getcode()
        if rescode == 200:
            response_body = response.read()
            response_json = json.loads(response_body)
            return pd.DataFrame(response_json['items'])
        else:
            print("Error Code:" + rescode)
            return pd.DataFrame()  # 빈 데이터프레임 반환
    except urllib.error.HTTPError as e:
        print(f"HTTP Error: {e.code} - {e.reason}")
        return pd.DataFrame()
    except urllib.error.URLError as e:
        print(f"URL Error: {e.reason}")
        return pd.DataFrame()

# HTML 태그(<b>)와 이모티콘, 특수문자 등을 제거하는 함수
def clean_text(text):
    if isinstance(text, str):
        # HTML 태그 제거
        text = re.sub(r'<.*?>', '', text)
        # 이모티콘 및 특수 기호 제거 (한글, 영어, 숫자, 공백 제외)
        text = re.sub(r'[^\w\s가-힣]', '', text)
    return text

 # 조사나 접속사, 이모티콘 등을 제거하는 함수
def clean_korean_text(text):
    if pd.isna(text):  # NaN 값일 경우 빈 문자열로 처리
        return ''
    
    # 2. 형태소 분석 후 조사, 접속사 제거
    morphs = okt.pos(text, norm=True, stem=True)
    
    # 3. 조사와 접속사 필터링 (조사(Josa), 접속사(Conjunction) 태그 제외)
    cleaned_text = ' '.join(word for word, tag in morphs if tag not in ['Josa', 'Conjunction', 'Adjective', 'Adverb', 'Eomi', 'Determiner', 'Exclamation', 'Suffix', 'Verb', 'KoreanParticle'])   

    return cleaned_text
    
# 청킹 함수: 명사(Noun), 형용사(Adjective) 등을 추출해 묶는 방식
def chunk_korean_text(text):
    if pd.isnull(text):  # text가 None인 경우 빈 문자열로 처리
        return ""
    
    english_words = re.findall(r'[a-zA-Z]+', text)
    
    morphs = okt.pos(text, norm=True, stem=True)
    
    filtered_words = [word for word, pos in morphs if pos in ['Noun', 'Alpha']]  # 명사와 영어 단어만 남김
    
    # 영어 단어 다시 추가
    filtered_words.extend(english_words)
    
    return ' '.join(filtered_words)

client_id = "내꺼"
client_secret = "내꺼"

# 모든 도시와 카테고리 파일을 한꺼번에 불러오는 코드
for city in cities:
    for category in categories:
        query = f'{city}{category}' ## 키워드 설정
        display = 100  # 최대값
        sort_options = ['sim', 'date']
        
        result_all = pd.DataFrame()
        total_results_per_sort = 1000  # 각 정렬 옵션 당 최대 1000개
        
        for sort in sort_options:
            for start in range(1, total_results_per_sort + 1, display):
                result = getresult(client_id, client_secret, query, display, start, sort)
                if not result.empty:
                    result_all = pd.concat([result_all, result], ignore_index=True)
                time.sleep(1)  # API 호출 간 딜레이 추가 (1초)

        # 기존 데이터프레임에서 title, description 컬럼만 선택하고 새로운 데이터프레임으로 만듦
        result = result_all[['title', 'description', 'postdate']].copy()
        
        # title과 description에서 HTML 태그 및 특수문자 제거
        result['title'] = result['title'].apply(clean_text)
        result['description'] = result['description'].apply(clean_text)
        
        # Konlpy의 Okt 객체 생성
        okt = Okt()

        # 1. content 컬럼에서 조사, 접속사, 이모티콘 제거 적용
        result['cleaned_description'] = result['description'].apply(lambda x: clean_korean_text(x))
        
        # 2. 조사, 접속사 제거된 텍스트에서 청킹 적용 (명사, 형용사, 동사만 남김)
        result['chunked_description'] = result['cleaned_description'].apply(lambda x: chunk_korean_text(x))

        # 3. CSV 파일로 저장
        output_file = f'{city}{category}_네이버_최종.csv'
        result[['title', 'chunked_description', 'postdate']].to_csv(output_file, index=False, encoding='utf-8-sig')

        print(f" '{output_file}' 파일로 저장 완료.")        

 '파리맛집_네이버_최종.csv' 파일로 저장 완료.
 '파리관광_네이버_최종.csv' 파일로 저장 완료.
 '파리핫플_네이버_최종.csv' 파일로 저장 완료.
 '파리숙소_네이버_최종.csv' 파일로 저장 완료.
 '파리쇼핑_네이버_최종.csv' 파일로 저장 완료.
 '파리여행_네이버_최종.csv' 파일로 저장 완료.
 '오사카맛집_네이버_최종.csv' 파일로 저장 완료.
 '오사카관광_네이버_최종.csv' 파일로 저장 완료.
 '오사카핫플_네이버_최종.csv' 파일로 저장 완료.
 '오사카숙소_네이버_최종.csv' 파일로 저장 완료.
 '오사카쇼핑_네이버_최종.csv' 파일로 저장 완료.
 '오사카여행_네이버_최종.csv' 파일로 저장 완료.
 '방콕맛집_네이버_최종.csv' 파일로 저장 완료.
 '방콕관광_네이버_최종.csv' 파일로 저장 완료.
 '방콕핫플_네이버_최종.csv' 파일로 저장 완료.
 '방콕숙소_네이버_최종.csv' 파일로 저장 완료.
 '방콕쇼핑_네이버_최종.csv' 파일로 저장 완료.
 '방콕여행_네이버_최종.csv' 파일로 저장 완료.
 '뉴욕맛집_네이버_최종.csv' 파일로 저장 완료.
 '뉴욕관광_네이버_최종.csv' 파일로 저장 완료.
 '뉴욕핫플_네이버_최종.csv' 파일로 저장 완료.
 '뉴욕숙소_네이버_최종.csv' 파일로 저장 완료.
 '뉴욕쇼핑_네이버_최종.csv' 파일로 저장 완료.
 '뉴욕여행_네이버_최종.csv' 파일로 저장 완료.
